
# Minería de datos sobre Twitter 3a

## Parte I: Extracción de twitters y replies por nombre de usuario y rango de fechas:
25 de Marzo

Fuentes:
    
https://medium.com/agatha-codes/0-to-280-getting-twitter-timeline-data-for-your-awesome-nlp-project-ff41b941ed6   
    
https://github.com/eleanorstrib/twitter_timeline_analysis_1/blob/master/twitter_timeline_to_csv.ipynb

https://towardsdatascience.com/mining-twitter-data-ba4e44e6aecc

https://github.com/owecodes/mining-twitter

https://gist.github.com/mchirico/01ab321cb3533908cac5874f1705e45d

Try to get replies to a particular set of tweets, recursively ...

Fuente: https://gist.github.com/edsu/54e6f7d63df3866a87a15aed17b51eaf

In [1]:
import tweepy
import sys
import datetime
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

consumer_key='koO4XqTuWFr5ADGcE8kjIkVoU'
consumer_secret='3F4sk9qU8zbKBROuLPUUj1uvE2YuhseXPe0ahMQoivg4icN5bL'
access_token='1230251564616515586-2KqPsCG2mIJp3irRjENgHpCfQUxTUg'
access_token_secret='6PJfMtYGY7w6csiIX9m1S5jFEKNZ3hE9PVkHKeN1S14iM'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Código que establece un radio y guarda en CSV

# agregar el despliegue de un mensaje en el caso de no 
# encontrar tweets según los para metros especificados

# Extraer tweets que contengan la palabra 
#IsladeMaipo dentro del 18 de octubre al 18 de marzo. [Considerar ubicación geográfica] 
tweet=''
import datetime
import csv
csvFile = open('tweet.csv', 'a')

filename = 'alcalde001_'+(datetime.datetime.now().strftime("%Y-%m-%d-%H"))+'.csv'

with open (filename, 'a+', newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in tweepy.Cursor(api.search , 
                   since='2019-10-18', q = 'coronavirus',
                               geocode="-33.746827,-70.900098,50km").items():
        
        
        print( str(tweet.text.encode('utf-8')), tweet.id, tweet.place)
        
       # print(str(j) +') '+ '\n' +
      #  'Texto del tweet: '+ i.full_text  '+ '\n'+ '\n')
        
        
        
        
        
        csvWriter.writerow([tweepy.created_at, 
                            tweepy.text.encode('utf-8'), 
                            tweepy.id, 
                            tweepy.place])
    if tweet == '':
        print('No hay nada que mostrar')

In [10]:
import os
import json
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from dotenv import load_dotenv
from flask import Flask, render_template
from threading import Thread
from shapely.geometry import mapping, shape
from flask_socketio import SocketIO
from flask_socketio import emit

load_dotenv()



# Server Config
app = Flask(__name__)
socketio = SocketIO(app)


def get_centroid(polygon):
    """
    params polygon: GeoJson polygon
    return: GoeJson Point
    """
    polygon = shape(polygon)
    point = polygon.centroid
    return mapping(point)

def parser(tweet):
    """
    params tweet: dict
    return: GeoJson dict
    """
    geoJson = {
        "type": "Feature",
        "properties": {
            "text": tweet['text'],
            "created_at": tweet['created_at'].split('+')[0],
            "user": "@" + tweet['user']['screen_name']
        },
        "geometry": get_centroid(tweet['place']['bounding_box']) 
        }

    return geoJson


class MyListener(StreamListener):
    def on_data(self, raw_data):
        data = json.loads(raw_data)
        try:
            if 'place' in data and data['place']:  # Only tweets with location info
                feature = parser(data)
                socketio.emit('addMarker', feature, json=True)  # Send the tweet to all connected clients
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

def start_filter(tags):
    twitter_stream = Stream(auth, MyListener())  # Create object for Twitter Streaming API
    twitter_stream.filter(track=tags) # Start getting all the tweets with the given tags

@app.route('/')
def home():
    return render_template('index.html')


if __name__ == '__main__':
    tags = ["#coronavirus"]   
    # Create and start thread for get the tweets
    Thread(target=start_filter, args=(tags,)).start()
    # Start the server
    socketio.run(app)

WebSocket transport not available. Install eventlet or gevent and gevent-websocket for improved performance.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
